<a href="https://colab.research.google.com/github/joserobertofox/datascience/blob/main/MVP2_DL_Vis%C3%A3o_Computacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP de Machine Learning & Analytics - Deep Learning - Visão Computacional**
## **Aluno: José Roberto Assis Silva**

## **1. Definição do Problema**

Este conjunto de dados foi criado com a finalidade de usá-lo em um drone e mapear/detectar doenças no campo de milho. Este conjunto de dados contém 4117 imagens de folhas doentes e saudáveis, distribuidas em 1550 doentes e 2567 saudáveis. Vamos criar um modelo que consiga mapear estas imagens e classificar.


In [127]:
#Importando as bibliotecas
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.preprocessing import image
import os
from sklearn.model_selection import train_test_split
import shutil
import random
import math
import numpy as np
import zipfile
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from datetime import datetime
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
import sklearn.metrics as skm
import itertools
import numpy as np
import matplotlib.pyplot as plt
import missingno as ms # para tratamento de missings
!pip install gdown
import gdown
import os
import zipfile

In [129]:
# Configuração para tratamento de erros
import warnings
warnings.filterwarnings("ignore")

In [130]:
#Função para imprimir a Matriz de Confusão
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Esta função imprime e plota a matriz de confusão.
    A normalização pode ser aplicada definindo `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusão normalizada")
    else:
        print('Matriz de confusão sem normalização')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Label real')
    plt.xlabel('Label predito')

## **2. Carga do Dataset**

Nesta etapa iremos um arquivo zipado que está no Google Drive e extrairemos a imagens de folhas doentes e folhas saudáveis. Posteriormente vamos criar um dataframe com as informações de cada imagem.

In [131]:
file_id = "1ETlFcANrOKUuTsZ_8aPuys_YUbxiXUg5"
url = f"https://drive.google.com/uc?id={file_id}"

folder_path = "./dataset"  # especifique a pasta onde baixara o arquivo
os.makedirs(folder_path, exist_ok=True) # cria a pasta de destino se necessário

output = "dataset/Corn.zip"  # especifique o nome do arquivo e a extensão do mesmo
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=1ETlFcANrOKUuTsZ_8aPuys_YUbxiXUg5
To: /content/dataset/Corn.zip
100%|██████████| 39.6M/39.6M [00:00<00:00, 118MB/s] 


'dataset/Corn.zip'

In [132]:
destination = 'dataset/images' # path onde se encontra o arquivo 'Corn.zip'

os.makedirs(destination,exist_ok=True) # cria a pasta de destino se necessário

with zipfile.ZipFile(output,'r') as zip_ref:
    zip_ref.extractall(destination)

In [133]:
r='dataset/images/Corn/diseased' # Verificando se as imagens foram extraídas na pasta diseased
print(os.listdir(r))

['919505a0-7e06-4a89-8faf-f308bd644a6a___RS_NLB 4141.JPG', 'J_170824_1318082.jpg', 'f0b49686-1a0f-4c01-9b60-e816c513215f___RS_NLB  4506 copy.jpg', 'ea9ba5b2-e8e7-48c8-ac16-d594a9e5cc51___RS_NLB  4443 copy 2.jpg', '824f1625-2e71-4df7-ab63-e0093d9a46a0___RS_NLB 3850.JPG', 'J_170824_1318076.jpg', '31a6d935-89b5-4feb-8fc6-44300ab3639e___RS_NLB 3640.JPG', 'J_170823_1344082.jpg', '6f370eac-7a90-40a3-be93-62147f32041c___RS_NLB  4608.JPG', 'J_170824_1329572.jpg', '5fc48a9b-7aee-4262-adf3-1102e354dd61___RS_NLB 3690.JPG', 'f546ba30-4b0b-4906-a527-bb4834df87cd___RS_NLB 3558.JPG', '95786afb-4b8e-4837-a396-0157ec1a2245___RS_NLB 0767.JPG', 'bf03fb8c-faeb-46e5-afe9-227289738dc4___RS_NLB 3749.JPG', 'J_170824_1318217.jpg', 'J_170823_1343430.jpg', 'f0edc304-8a96-4b71-bf5b-2a379967ba27___RS_NLB 4018.JPG', 'J_170823_1346463.jpg', 'J_170824_1318530.jpg', 'J_170824_1331013.jpg', 'b7fc574e-e12e-4dc5-94c2-698892884e66___RS_NLB 0817.JPG', 'J_170823_1346334.jpg', '70cfac48-3bf0-4134-984a-3e9a19e2cdf0___RS_NLB 4

In [134]:
r='dataset/images/Corn/healthy' # Verificando se as imagens foram extraídas na pasta healthy
print(os.listdir(r))

['3ed3e07e-1932-4c33-9613-d31af970986c___R.S_HL 7973 copy.jpg', 'J_170824_1331134.jpg', 'J_170824_1318232.jpg', 'J_170824_1329570.jpg', 'J_170824_1330119.jpg', 'J_170824_1330318.jpg', 'J_170824_1330392.jpg', '25c17c68-5641-473f-a052-9335b1376557___R.S_HL 8202 copy.jpg', 'J_170824_13192713.jpg', 'J_170824_1328278.jpg', 'J_170824_1319215.jpg', '8c3c8212-cfa7-4e17-8c88-c3b5d274889f___R.S_HL 8208 copy.jpg', 'J_170824_13184416.jpg', '5face5cd-e150-42f5-8ae6-eef79c43b05b___R.S_HL 8010 copy 2.jpg', 'J_170824_1319142.jpg', 'J_170824_13184212.jpg', '19f6234f-30e6-41e1-90bf-8204612bb150___R.S_HL 8223 copy.jpg', 'J_170824_1330115.jpg', 'J_170823_1346468.jpg', '8b6204cd-cbf2-43b0-90cc-255815f7beb7___R.S_HL 8156 copy.jpg', 'ae2d73a4-c6e2-4b85-aaa7-3027e2a2c6a9___R.S_HL 5564 copy.jpg', 'c68a8c80-fe05-4fd5-a9dc-d9e746f86519___R.S_HL 7966 copy.jpg', '5467efda-f4ec-4f5e-94b8-8bc0fd19c7ac___R.S_HL 8148 copy.jpg', 'ca005f55-ec22-41ef-b162-66724347da83___R.S_HL 5518 copy.jpg', 'J_170824_13192012.jpg', 'J_

In [135]:
path='dataset/images/Corn/diseased/' # Pasta onde estão as de imagens de folhas doentes
path1='dataset/images/Corn/healthy/' # Pasta onde estão as de imagens de folhas saudáveis
print(f"Temos um total de {len(os.listdir(path))} imagens de folhas de milho doentes para o treinamento e avaliação do modelo de deep learning")
print(f"Temos um total de {len(os.listdir(path1))} imagens de folhas de milho saudáveis para o treinamento e avaliação do modelo de deep learning")
print(f"Temos um total de {len(os.listdir(path))+len(os.listdir(path1))} imagens de folhas de milho para o treinamento e avaliação do modelo de deep learning")

Temos um total de 1550 imagens de folhas de milho doentes para o treinamento e avaliação do modelo de deep learning
Temos um total de 2567 imagens de folhas de milho saudáveis para o treinamento e avaliação do modelo de deep learning
Temos um total de 4117 imagens de folhas de milho para o treinamento e avaliação do modelo de deep learning


In [136]:
print(f"Primeiras 20 imagens de folhas de milho doentes: \n {os.listdir(path)[:20]}") # Listando as 20 primeiras imagens
print(f"Primeiras 20 imagens de folhas de milho saudáveis: \n {os.listdir(path1)[:20]}") # Listando as 20 primeiras imagens

Primeiras 20 imagens de folhas de milho doentes: 
 ['919505a0-7e06-4a89-8faf-f308bd644a6a___RS_NLB 4141.JPG', 'J_170824_1318082.jpg', 'f0b49686-1a0f-4c01-9b60-e816c513215f___RS_NLB  4506 copy.jpg', 'ea9ba5b2-e8e7-48c8-ac16-d594a9e5cc51___RS_NLB  4443 copy 2.jpg', '824f1625-2e71-4df7-ab63-e0093d9a46a0___RS_NLB 3850.JPG', 'J_170824_1318076.jpg', '31a6d935-89b5-4feb-8fc6-44300ab3639e___RS_NLB 3640.JPG', 'J_170823_1344082.jpg', '6f370eac-7a90-40a3-be93-62147f32041c___RS_NLB  4608.JPG', 'J_170824_1329572.jpg', '5fc48a9b-7aee-4262-adf3-1102e354dd61___RS_NLB 3690.JPG', 'f546ba30-4b0b-4906-a527-bb4834df87cd___RS_NLB 3558.JPG', '95786afb-4b8e-4837-a396-0157ec1a2245___RS_NLB 0767.JPG', 'bf03fb8c-faeb-46e5-afe9-227289738dc4___RS_NLB 3749.JPG', 'J_170824_1318217.jpg', 'J_170823_1343430.jpg', 'f0edc304-8a96-4b71-bf5b-2a379967ba27___RS_NLB 4018.JPG', 'J_170823_1346463.jpg', 'J_170824_1318530.jpg', 'J_170824_1331013.jpg']
Primeiras 20 imagens de folhas de milho saudáveis: 
 ['3ed3e07e-1932-4c33-9613-

In [137]:
images =os.listdir(path)

labels, heights, widths, channels, filesize = [], [], [], [], []

# Loop que percorre cada uma das imagens de folhas doentes extraídas
for fname in images:

    labels.append("dis")
    img_shape = mpimg.imread(path+fname).shape
    heights.append(img_shape[0])
    widths.append(img_shape[1])
    channels.append(img_shape[2])
    filesize.append(os.path.getsize(path+fname))

# Criação do dataset
df = pd.DataFrame({'images': images, 'class': labels, 'height': heights,
                         'width': widths, 'channels': channels, 'filesize': filesize})

In [126]:
images1 =os.listdir(path1)

labels1, heights1, widths1, channels1, filesize1 = [], [], [], [], []

# Loop que percorre cada uma das imagens de folhas saudáveis extraídas
for fname in images1:

    labels1.append("hea")
    img_shape = mpimg.imread(path1+fname).shape
    heights1.append(img_shape[0])
    widths1.append(img_shape[1])
    channels1.append(img_shape[2])
    filesize1.append(os.path.getsize(path1+fname))

# Criação do dataset
df1 = pd.DataFrame({'images': images1, 'class': labels, 'height': heights,
                          'width': widths, 'channels': channels, 'filesize': filesize})


ValueError: ignored

In [110]:
df.head(20)

,images,class,height,width,channels,filesize
0,919505a0-7e06-4a89-8faf-f308bd644a6a___RS_NLB ...,dis,256,256,3,13841
1,J_170824_1318082.jpg,dis,256,256,3,8638
2,f0b49686-1a0f-4c01-9b60-e816c513215f___RS_NLB ...,dis,256,256,3,11817
3,ea9ba5b2-e8e7-48c8-ac16-d594a9e5cc51___RS_NLB ...,dis,256,256,3,15640
4,824f1625-2e71-4df7-ab63-e0093d9a46a0___RS_NLB ...,dis,256,256,3,13046
5,J_170824_1318076.jpg,dis,256,256,3,6889
6,31a6d935-89b5-4feb-8fc6-44300ab3639e___RS_NLB ...,dis,256,256,3,14832
7,J_170823_1344082.jpg,dis,256,256,3,6741
8,6f370eac-7a90-40a3-be93-62147f32041c___RS_NLB ...,dis,256,256,3,12636
9,J_170824_1329572.jpg,dis,256,256,3,5781
